## Imports

In [1]:
import pandas as pd
import urllib.parse , requests
import numpy as np
from IPython.display import HTML
import base64  

## Main function

In [5]:
def geonames_iri(placename):
    iri = ''
    if isinstance(placename,str):
        geonames_url = 'http://api.geonames.org/search?q='+urllib.parse.quote(placename)+'&fuzzy=0.8&username=palread&type=json'
        r = requests.get(geonames_url)
        if r.status_code == 200:
            data = r.json()
            iri = "http://www.geonames.org/"+str(data["geonames"][0]["geonameId"])
    
    return iri

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv(index =False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)



# read an online csv, get the specified column 
def add_geonames(url, geonames_col):
    separator ='\t' if 'tsv' in url else ','
    df = pd.read_csv(url, error_bad_lines=False, sep=separator, header=0 )
    # query geonames
    places = df[geonames_col].tolist()
    geonames = [geonames_iri(place) for place in places]

    # return a new column with the geonames IDs
    df[geonames_col+'_geonames'] = geonames

    # export csv
    return create_download_link(df)

In [6]:
# change url of the table, and the name of the column including places
url = 'https://raw.githubusercontent.com/palread/import_csv/main/alsharekh%20magazine%20archive%20-%20author_sample.tsv?token=ABRE777S3SL6HYO4DW3YGBTAN44HE'
geonames_col = 'geonames'

# run
add_geonames(url, geonames_col)